In [1]:
import sys
sys.path.append("../library")

In [2]:
from thinkbayes2 import Suite

In [3]:
# dice puzzle: based on sampling result of dice, to get the most possible dice
class Dice(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
#             impossible for dice 'hypo' throw out number 'data'
            return 0
        else:
            return 1.0 / hypo

In [4]:
suite = Dice([4, 6, 8, 12, 20])
suite.Update(6)
suite.Print()

4 0.0
6 0.3921568627450981
8 0.29411764705882354
12 0.19607843137254904
20 0.11764705882352944


In [5]:
suite = Dice([4, 6, 8, 12, 20])
suite.UpdateSet([6, 8, 7, 7, 5, 4])
suite.Print()

4 0.0
6 0.0
8 0.91584527196901
12 0.08040342579700496
20 0.0037513022339850668


In [3]:
# locomotive problem from Freadrick Mosteller's fifty challenging problems in probablity with solution (Dover, 1987)
class Train(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
        else:
            return 1.0 / hypo

In [6]:
N=1000
hypos=range(1, 1+N)
suite=Train(hypos)
suite.Update(60)
# suite.Print()

1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.005905417875729855
61 0.00580860774661953
62 0.00571492052489986
63 0.0056242075006951
64 0.005536329258496739
65 0.005451154962212174
66 0.00536856170520896
67 0.005288433918564049
68 0.005210662831526343
69 0.005135145978895526
70 0.00506178675062559
71 0.004990493979490018
72 0.004921181563108212
73 0.004853768117038236
74 0.00478817665599718
75 0.004724334300583884
76 0.004662172007155148
77 0.004601624318750537
78 0.004542629135176811
79 0.00448512750055432
80 0.004429063406797391
81 0.004374383611651744
82 0.004321037470046235
83 0.0042689767776360405
84 0.004218155625521325
85 

727 0.0004873797421510197
728 0.00048671026448322987
729 0.00048604262351686044
730 0.0004853768117038237
731 0.00048471282153733414
732 0.0004840506455516275
733 0.00048339027632167983
734 0.00048273170646293093
735 0.00048207492863100855
736 0.00048141993552145557
737 0.00048076671986945895
738 0.0004801152744495817
739 0.0004794655920754957
740 0.000478817665599718
741 0.00047817148791334856
742 0.00047752705194581044
743 0.00047688435066459123
744 0.00047624337707498837
745 0.0004756041242198541
746 0.00047496658517934493
747 0.00047433075307067106
748 0.0004736966210478493
749 0.000473064182301457
750 0.00047243343005838845
751 0.0004718043575816129
752 0.00047117695816993523
753 0.00047055122515775734
754 0.00046992715191484255
755 0.00046930473184608116
756 0.00046868395839125835
757 0.00046806482502482344
758 0.00046744732525566134
759 0.000466831452626866
760 0.00046621720071551485
761 0.0004656045631324459
762 0.0004649935335220358
763 0.00046438410556198074
764 0.00046377627